## Positive samples with GMM

In [ ]:
gmm_data = np.array([w[:,:-1,:].flatten() for w in wake_samples])

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(gmm_data)
data.shape

In [ ]:
n_components = np.arange(1, 20)
models = [GMM(n, covariance_type='full', random_state=0)
          for n in n_components]
aics = [model.fit(data).aic(data) for model in models]
plt.plot(n_components, aics);

In [ ]:
gmm = GMM(5, covariance_type='full', random_state=0)
gmm.fit(data)

In [ ]:
data_new = gmm.sample(100)
data_new[0].shape

In [ ]:
wakes_new = pca.inverse_transform(data_new[0])
wakes_new.shape

In [ ]:
wakes_new = wakes_new.reshape(100,4, 28, 60)

In [ ]:
wp.make_12_heatmaps(wakes_new[:12,3,:,:])

## Autoencoder 1

In [ ]:
IMAGE_SIZE = 1680
IMAGE_WIDTH = 60
IMAGE_HEIGHT = 28
code_size = 200
num_epochs = 10

lr = 0.002
optimizer_cls = optim.Adam

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, code_size):
        super().__init__()
        self.code_size = code_size
        
        # Encoder
        self.enc_cnn_1 = nn.Conv2d(1, 10, kernel_size=5)
        self.enc_cnn_2 = nn.Conv2d(10, 20, kernel_size=5)
        self.enc_linear_1 = nn.Linear(4 * 12 * 20, 50)
        self.enc_linear_2 = nn.Linear(50, self.code_size)
        
        # Decoder
        self.dec_linear_1 = nn.Linear(self.code_size, 160)
        self.dec_linear_2 = nn.Linear(160, IMAGE_SIZE)
        
    def forward(self, images):
        code = self.encode(images)
        out = self.decode(code)
        return out, code
    
    def encode(self, images):
        code = self.enc_cnn_1(images)
        code = F.selu(F.max_pool2d(code, 2))
        
        code = self.enc_cnn_2(code)
        code = F.selu(F.max_pool2d(code, 2))
        
        code = code.view([images.size(0), -1])
        code = F.selu(self.enc_linear_1(code))
        code = self.enc_linear_2(code)
        return code
    
    
    def decode(self, code):
        
        out = F.selu(self.dec_linear_1(code))
        out = torch.sigmoid(self.dec_linear_2(out))

        out = out.view([code.size(0), 1, IMAGE_HEIGHT, IMAGE_WIDTH])
        return out

In [ ]:
autoencoder = Autoencoder()
loss_fn = nn.MCE()
optimizer = optimizer_cls(autoencoder.parameters(), lr=lr)
autoencoder  = autoencoder.double()
for epoch in range(num_epochs):
    print("Epoch %d" % epoch)
    
    for i, images in enumerate(train_loader):    # Ignore image labels
        out = autoencoder(images.unsqueeze(1))
        loss = loss_fn(out, images.unsqueeze(1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print("Loss = %.3f" % loss.item())

## Early CNN Attempts

In [ ]:
# YOUR CODE HERE
class CNN1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5)
        self.fc1 = nn.Linear(50*434*4, 100)
        self.fc2 = nn.Linear(100, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2)
        print(x.shape)
        x = x.view(-1, 50*434*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = CNN1().to('cpu')
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:

batch_size = 1
data_in = torch.Tensor(data_in)
data_out = torch.Tensor(data_out)
dataset = TensorDataset(data_in, data_out)
#train_dataset, val_dataset = random_split(dataset, [int(np.ceil(len(data_in)*0.99)), int(np.floor(len(data_in)*0.01))])
train_data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
data_in.view(1, 1, 1750, 28)

In [ ]:
model(data_in.view(1, 1, 1750, 28))

In [ ]:
count = 0
for epoch in range(1):
    for i, (batch_x, batch_y) in enumerate(train_data_loader):
        # Put data in the correct device
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device).long()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(batch_x.view(batch_size,1,1750, 28))
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, batch_y)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        count += 1

        #if count % 5000 == 0:
    correct = 0
    total = 0
    v_loss = 0

#     for val_x, val_y in val_data_loader:

#         # Put data in the correct device
#         val_x = val_x.to(device)
#         val_y = val_y.to(device).long()
#         # Forward pass only to get logits/output
#         with torch.no_grad():
#             output = model(val_x.view(batch_size,1, seq_len))

#         # Get predictions from the maximum value
#         _, predicted = torch.max(output, 1)
#         val_bloss = criterion(output, val_y)
#         v_loss += val_bloss*batch_size

#         # Total correct predictions
#         total += batch_size
#         correct += (predicted == val_y).sum()
#    accuracy = 100 * correct / total
#    v_loss = v_loss/total

    # Print Loss
#    print('Epoch: {}. Loss: {}. ValLoss: {}. Accuracy: {} %'.format(epoch, loss.item(), v_loss, accuracy))
    print('Epoch: {}. Loss: {}. %'.format(epoch, loss.item()))